# Sir_ChatGPT

Development of a Simple Informational Retrieval System obtaining guidance with ChatGPT. Tested on CISI collection with BM25 algorithm.


Autor: Marcus Vinícius Borela de Castro


[Repositório no github](https://github.com/marcusborela/Sir-ChatGPT)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcusborela/Sir-ChatGPT/blob/main/code/Sir_ChatGPT.ipynb)

## Rotinas utilitárias

In [8]:
def mostra_dict(dicionario: dict):
    """
    Imprime informações sobre o dicionário recebido como parâmetro.

    Argumentos:
    - dicionario: um dicionário a ser impresso

    Retorna:
    - None
    """
    # obtém a primeira e última chave do dicionário
    primeiro_elemento = list(dicionario.keys())[0]
    ultimo_elemento = list(dicionario.keys())[-1]

    # imprime o tamanho do dicionário e as informações sobre seus limites
    print(f"O dicionário tem tamanho: {len(dicionario)}")
    print(f"Seus limites:\n {primeiro_elemento}:\n {dicionario[primeiro_elemento]},\n {ultimo_elemento}:\n {dicionario[ultimo_elemento]}")


# Etapa 1: Coleta dos dados

## Baixar CISI Collection

In [9]:
# import requests
# import io

import os
import urllib.request
import tarfile
from collections import defaultdict


In [10]:

# Define a URL da coleção CISI
url = "http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz"

# Define o caminho onde o arquivo será salvo
file_path = "cisi.tar"

# Faz o download do arquivo
urllib.request.urlretrieve(url, file_path)

# Extrai os arquivos da coleção CISI
with tarfile.open(file_path, "r") as tar:
    tar.extractall()


In [12]:
os.remove('cisi.tar')

Rotina para carregar arquivos da CISI Collection

Os arquivos com consultas e documentos têm um formato padrão:
.I <id>
.tag
valor tag

E as tags podem ser:

    .T = "title"
    .A = "author"
    .X = "reference"
    .B = "bibliograph"
    .W = "text"

In [14]:
def carregar_cisi_arquivo(nome_arquivo:str, se_debug:bool=False):

  # abre o arquivo CISI.ALL em modo de leitura
  with open(nome_arquivo, "r") as f:
      lines = f.readlines()

  # inicializa o dicionário
  documents = {}

  # inicializa as variáveis de controle
  doc_id = None
  field = None
  text = ""

  # percorre as linhas do arquivo
  for line in lines:
    # se a linha começa com ".I", então é um novo documento
    if line.startswith((".I", ".T", ".A", ".W", ".X", ".B")):
      # faça algo se a linha começar com um desses valores
        if doc_id is not None and field != "id":
            documents[doc_id][field] = text.strip()
            if se_debug:
              print(f"Atribuido a documents[{doc_id}][{field}] = {text.strip()}")
            text = ""
    # se a linha começa com ".I", então é um novo documento
    if line.startswith(".I"):
        # extrai o ID do documento da linha
        doc_id = int(line.split()[1])
        field = "id"
        documents[doc_id] = {}
        if se_debug:
          print(f"novo doc_id {doc_id}")
    # se a linha começa com ".T", ".A" ou ".W", então é um novo campo
    elif line.startswith(".T"):
        field = "title"
    elif line.startswith(".A"):
        field = "author"
    elif line.startswith(".X"):
        field = "reference"
    elif line.startswith(".B"):
        field = "bibliograph"
    elif line.startswith(".W"):
        field = "text"
    # caso contrário, é um texto que faz parte do campo atual
    else:
        text += line
    if se_debug:
      print(f'doc_id {doc_id} field {field}: {line}')
      if doc_id == 4: break 

  # adiciona o último documento ao dicionário
  if doc_id is not None:
      documents[doc_id][field] = text.strip()

  return documents

### Carregando Documentos - CISI.ALL

In [15]:
documentos = carregar_cisi_arquivo('CISI.ALL')

In [16]:
mostra_dict(documentos)

O dicionário tem tamanho: 1460
Seus limites:
 1:
 {'title': '18 Editions of the Dewey Decimal Classifications', 'author': 'Comaromi, J.P.', 'text': "The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.", 'reference': '1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1'},
 1460:
 {'title': 'Modern Integral Information Systems for Chemistry and Chemical Technology', 'author': 'Chernyi, A.I.', 'text': "At the present time, about 15% of all the world publications of \nscientific and

### Carregando Consultas - CISI.QRY

In [17]:
consultas = carregar_cisi_arquivo('CISI.QRY')

In [18]:
mostra_dict(consultas)

O dicionário tem tamanho: 112
Seus limites:
 1:
 {'text': 'What problems and concerns are there in making up descriptive titles?\nWhat difficulties are involved in automatically retrieving articles from\napproximate titles?\nWhat is the usual relevance of the content of articles to their titles?'},
 112:
 {'title': 'A Fast Procedure for the Calculation of Similarity Coefficients in\nin Automatic Classification', 'author': 'Willett, P.', 'text': 'A fast algorithm is described for comparing the lists of terms representing\ndocuments in automatic classification experiments.  The speed of the procedure\narises from the fact that all of the non-zero-valued coefficicents for a given\ndocument are identified together, using an inverted file to the terms in the\ndocument collection.  The complexity and running time of the algorithm are\ncompared with previously described procedures.', 'bibliograph': '(Information Processing & Management, Vol. 17, No. 2, 1981, pp. 53-60)'}


### Carregando dados de relevância - CISI.REL

In [19]:

# inicializa o dicionário de relevância por consulta
relevancia_consulta = {}

# abre o arquivo CISI.REL em modo de leitura
with open('CISI.REL') as f:
    # percorre as linhas do arquivo
    for line in f.readlines():
        # extrai o ID da consulta e do documento da linha
        qry_id = int(line.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0])
        doc_id = int(line.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])
        
        # adiciona o ID do documento na lista de relevância da consulta
        if qry_id in relevancia_consulta:
            relevancia_consulta[qry_id].append(doc_id)
        else:
            relevancia_consulta[qry_id] = []
            relevancia_consulta[qry_id].append(doc_id)
# Fonte de apoio: https://www.kaggle.com/code/razamh/some-changes-in-cisi-eda

In [20]:
mostra_dict(relevancia_consulta)

O dicionário tem tamanho: 76
Seus limites:
 1:
 [28, 35, 38, 42, 43, 52, 65, 76, 86, 150, 189, 192, 193, 195, 215, 269, 291, 320, 429, 465, 466, 482, 483, 510, 524, 541, 576, 582, 589, 603, 650, 680, 711, 722, 726, 783, 813, 820, 868, 869, 894, 1162, 1164, 1195, 1196, 1281],
 111:
 [328, 422, 448, 485, 503, 509]


from google.colab import drive
drive.mount('/content/drive')

# Etapa 2: Pré-processamento dos textos

In [ ]:
# !pip install nltk

In [21]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
#stemmer = SnowballStemmer('english')


In [22]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
stop_words = set(stopwords.words('english'))

### Código do pré-processador

In [40]:
def preprocessa_texto(texto,
                      to_lower=True, 
                      remove_pontuacao=True,
                      remove_stopwords=True,
                      aplica_stemming=True,
                      aplica_lematizacao=True):
    """
    Função que realiza o pré-processamento de um texto.
    
    Parâmetros:
    texto (str): Texto a ser pré-processado
    to_lower (bool): Flag que indica se deve transformar o texto para lower case. Default: True
    remove_pontuacao (bool): Flag que indica se deve remover a pontuação do texto. Default: True
    remove_stopwords (bool): Flag que indica se deve remover as stop words do texto. Default: True
    aplica_stemming (bool): Flag que indica se deve aplicar stemming no texto. Default: True
    aplica_lematizacao (bool): Flag que indica se deve aplicar lematização no texto. Default: True
    
    Retorna:
    str: Texto pré-processado
    """
    # Transforma o texto em lower case
    if to_lower:
        texto = texto.lower()

    # Substitute line breaks for space
    texto = re.sub(r'\n', ' ', texto)

    # Remove pontuação
    if remove_pontuacao:
        texto = re.sub(r'[^\w\s]', '', texto)

    palavras = texto.split()

    # Remove stop words
    if remove_stopwords:
        palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stop_words]
        palavras = palavras_sem_stopwords

    # Aplica stemming
    if aplica_stemming:
        palavras_stemizadas = [stemmer.stem(palavra) for palavra in palavras]
        palavras = palavras_stemizadas

    # Aplica lematização
    if aplica_lematizacao:
        palavras_lematizadas = [lemmatizer.lemmatize(palavra) for palavra in palavras]
        palavras = palavras_lematizadas

    return ' '.join(palavras)


In [41]:
preprocessa_texto("This is an example of text.") 

'exampl text'

In [29]:
def preprocessa_texto_em_dict(parm_dict, 
                      to_lower=True, 
                      remove_pontuacao=True,
                      remove_stopwords=True,
                      aplica_stemming=True,
                      aplica_lematizacao=True):
  """
  Recebe um dicionário e retorna uma cópia do mesmo com os valores da chave "text" pré-processados.

  Args:
  parm_dict (dict): Dicionário com chaves de texto e valores em texto.
  to_lower (bool): Transforma o texto em caixa baixa. Padrão é True.
  remove_pontuacao (bool): Remove a pontuação do texto. Padrão é True.
  remove_stopwords (bool): Remove as palavras de parada do texto. Padrão é True.
  aplica_stemming (bool): Aplica a técnica de stemming no texto. Padrão é True.
  aplica_lematizacao (bool): Aplica a técnica de lematização no texto. Padrão é True.

  Returns:
  dict: Novo dicionário com a nova chave 'texto_prep' e seus valores pré-processados.

  new_dict = dict(parm_dict)  # cria uma cópia do dicionário
  for elemento in new_dict:
    for key in new_dict[elemento]:
      if key == "text":
          new_dict[elemento]["text_prep"] = preprocessa_texto(new_dict[elemento]["text"], to_lower=to_lower, 
                                                    remove_pontuacao=remove_pontuacao,
                                                    remove_stopwords=remove_stopwords,
                                                    aplica_stemming=aplica_stemming,
                                                    aplica_lematizacao=aplica_lematizacao)
  return new_dict
  """
  new_dict = {}
  for elemento in parm_dict:
    new_dict[elemento] = parm_dict[elemento].copy()
    for key in parm_dict[elemento]:
      if key == "text":
          new_dict[elemento]["text_prep"] = preprocessa_texto(new_dict[elemento]["text"], to_lower=to_lower, 
                                                    remove_pontuacao=remove_pontuacao,
                                                    remove_stopwords=remove_stopwords,
                                                    aplica_stemming=aplica_stemming,
                                                    aplica_lematizacao=aplica_lematizacao)
  return new_dict

### Teste do código de pre-processamento

In [42]:
assert preprocessa_texto("This is a simple text.").split() == ['simpl', 'text']


In [43]:
assert preprocessa_texto("Hello! My name is John. Nice to meet you!").split() == ['hello', 'name', 'john', 'nice', 'meet']
assert preprocessa_texto("We are learning about Natural Language Processing.").split() == ['learn', 'natur', 'languag', 'process']
assert preprocessa_texto("The quick brown fox jumps over the lazy dog.").split() == ['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']
assert preprocessa_texto("To be, or not to be: that is the question.").split() == ["question"]
assert preprocessa_texto("I'm a developer at OpenAI. I love working with AI and NLP technologies!").split() == ['im', 'develop', 'openai', 'love', 'work', 'ai', 'nlp', 'technolog']
assert preprocessa_texto("The cat is on the mat.").split() == ['cat', 'mat']
assert preprocessa_texto("An investment in knowledge pays the best interest.").split() == ['invest', 'knowledg', 'pay', 'best', 'interest']
assert preprocessa_texto("The quick brown fox jumps over the lazy dog.").split() == ['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']
assert preprocessa_texto("To be, or not to be: that is the question.").split() == ["question"]
assert preprocessa_texto("I'm a developer at OpenAI. I love working with AI and NLP technologies!").split() == ['im', 'develop', 'openai', 'love', 'work', 'ai', 'nlp', 'technolog']
assert preprocessa_texto("The cat is on the mat.").split() == ['cat', 'mat']
assert preprocessa_texto("An investment in knowledge pays the best interest.").split() == ['invest', 'knowledg', 'pay', 'best', 'interest']


In [44]:
assert preprocessa_texto("Hello World!").split() == ['hello', 'world']
assert preprocessa_texto("Hello, World!!!").split() == ['hello', 'world']
assert preprocessa_texto("Hello World", remove_pontuacao=False).split() == ['hello', 'world']
assert preprocessa_texto("Hello World", remove_stopwords=False).split() == ['hello', 'world']
assert preprocessa_texto("I am running in the park", aplica_stemming=False).split() == ['running', 'park']
assert preprocessa_texto("I am running in the park", to_lower=False, remove_stopwords=False, aplica_stemming=False, aplica_lematizacao=False).split() == ['I', 'am', 'running', 'in', 'the', 'park']
assert preprocessa_texto("I am running in the park", aplica_lematizacao=False).split() == ['run', 'park']
assert preprocessa_texto("I am running in the park", to_lower=False).split() == ['i', 'run', 'park']


In [45]:
documentos[1]

{'title': '18 Editions of the Dewey Decimal Classifications',
 'author': 'Comaromi, J.P.',
 'text': "The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.",
 'reference': '1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1'}

### Pré-processamento da coleção

In [46]:
consultas_prep = preprocessa_texto_em_dict(consultas)

In [47]:
consultas_prep[1]

{'text': 'What problems and concerns are there in making up descriptive titles?\nWhat difficulties are involved in automatically retrieving articles from\napproximate titles?\nWhat is the usual relevance of the content of articles to their titles?',
 'text_prep': 'problem concern make descript titl difficulti involv automat retriev articl approxim titl usual relev content articl titl'}

In [48]:
documentos_prep = preprocessa_texto_em_dict(documentos)

In [49]:
documentos_prep[1]

{'title': '18 Editions of the Dewey Decimal Classifications',
 'author': 'Comaromi, J.P.',
 'text': "The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.",
 'reference': '1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1',
 'text_prep': 'present studi histori dewey decim classif first edit ddc publish 1876 eighteenth edit 1971 futur edit continu appear need spite ddc long healthi life howev full stori never told biographi dewey briefli describ system first attempt provid detail

# Etapa 3: Indexação

In [34]:
!pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
from rank_bm25 import BM25Okapi


In [64]:
import math
from collections import Counter

class BM25:
    
    def __init__(self, documents, k1=1.2, b=0.75, epsilon=0.25):
        """
        Inicializa a classe BM25 com os documentos a serem indexados e os parâmetros do modelo.

        Args:
        documents (dict): Dicionário com chaves de texto e valores em texto pré-processado.
        k1 (float): Parâmetro de ajuste para a frequência do termo (padrão é 1,2).
        b (float): Parâmetro de ajuste para o comprimento do documento (padrão é 0,75).
        epsilon (float): Parâmetro de suavização para evitar divisão por zero (padrão é 0,25).
        """
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        self.N = len(documents)
        self.avgdl = sum([len(documents[doc]["text_prep"]) for doc in documents]) / self.N
        self.f = []  # Lista vazia que será preenchida com as frequências dos termos para cada documento
        self.df = {}  # Dicionário vazio que será preenchido com o número de documentos que contêm cada termo
        self.idf = {}  # Dicionário vazio que será preenchido com os valores IDF de cada termo
        self.documents = documents
        self.criar_index()  # Cria o índice invertido dos documentos fornecidos
    
    def criar_index(self):
        """
        Realiza a indexação dos documentos fornecidos no construtor da classe.
        """
        for doc in self.documents:
            frequencies = Counter(self.documents[doc]["text_prep"])  # Frequências dos termos no documento
            document_length = len(self.documents[doc]["text_prep"])  # Comprimento do documento em termos de palavras
            self.f.append(frequencies)  # Adiciona a frequência dos termos do documento na lista self.f
            for word in frequencies:
                if word not in self.df:
                    self.df[word] = 1
                else:
                    self.df[word] += 1  # Atualiza o número de documentos que contêm o termo
            for word, freq in frequencies.items():
                if word not in self.idf:
                    self.idf[word] = math.log((self.N - self.df[word] + 0.5) / (self.df[word] + 0.5))
                    # Calcula e armazena o IDF de cada termo
            #não gerou segunda vez: self.score(doc, document_length)

    def score(self, document_id, document_length):
        """
        Calcula o score BM25 para um documento.

        Args:
        document_id (str): ID do documento a ser calculado o score.
        document_length (int): Comprimento do documento em termos de palavras.

        Returns:
        float: O score BM25 para o documento.
        """
        score = 0  # Inicializa o score do documento com zero
        for word in self.f[-1]:
            freq = self.f[-1][word]
            idf = self.idf[word]
            num = idf * freq * (self.k1 + 1)
            denom = freq + self.k1 * (1 - self.b + self.b * document_length / self.avgdl)
            score += num / denom  # Calcula e adiciona o score do term
        return score

    def buscar(self, query, n=10):
        """
        Realiza a busca de documentos utilizando a query fornecida.

        Args:
        query (str): A query de busca.
        n (int): O número de documentos a serem retornados.

        Returns:
        list: Uma lista de tuplas com o ID do documento e seu score BM25 correspondente.
        """
        query = query.split()
        query = [word for word in query if word in self.idf.keys()]
        query_freq = Counter(query)
        query_weights = {}
        for query_term, query_term_freq in query_freq.items():
            idf = self.idf[query_term]
            query_weights[query_term] = idf * query_term_freq * (self.k1 + 1) / (query_term_freq + self.epsilon)

        scores = []
        for document_id in self.documents:
            document_length = len(self.documents[document_id]["text_prep"])
            score = self.score(document_id, document_length)
            doc_weights = self.f[-1]
            for query_term, query_term_weight in query_weights.items():
                if query_term in doc_weights:
                    score += query_term_weight * (self.k1 + 1) * doc_weights[query_term] / (doc_weights[query_term] + self.k1 * (1 - self.b + self.b * document_length / self.avgdl))
            scores.append((document_id, score))

        return sorted(scores, key=lambda x: x[1], reverse=True)[:n]            


In [60]:
class Indexador:
    def __init__(self, documentos_prep):
        """
        Inicializa o indexador.

        Args:
            documentos_prep (dict): Dicionário com as informações dos documentos pré-processados.
        """
        self.documentos_prep = documentos_prep
        self.index = None
        self.bm25 = None
        self.doc_ids = None

    def criar_index(self):
        """
        Cria o índice invertido dos documentos.
        """
        docs = [doc["text_prep"] for doc in self.documentos_prep.values()]
        self.index = BM25Okapi(docs)
        self.doc_ids = list(self.documentos_prep.keys())

    def buscar(self, consulta, top_k=10):
        """
        Realiza a busca BM25 para a consulta.

        Args:
            consulta (str): Consulta a ser buscada.
            top_k (int): Número máximo de documentos a serem retornados. Padrão é 10.

        Returns:
            list: Lista com os ids dos documentos mais relevantes para a consulta.
        """
        if self.index is None:
            self.criar_index()

        tokenized_consulta = consulta.split(" ")
        scores = self.index.get_scores(tokenized_consulta)
        top_k_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        return [self.doc_ids[i] for i in top_k_indices]



In [37]:
index_bm25 = Indexador(documentos_prep)

In [38]:
index_bm25.criar_index()

In [50]:
index_bm25.buscar(consultas_prep[1]['text_prep'])

[429, 722, 1299, 759, 65, 76, 38, 711, 603, 820]

In [65]:
index_bm25 = BM25(documentos_prep)

In [66]:
index_bm25.buscar(consultas_prep[1]['text_prep'])

[(1288, 391.6020621373468),
 (1296, 391.4578213881378),
 (1284, 389.46340036696995),
 (1086, 388.6225558262266),
 (1301, 388.6225558262266),
 (1289, 388.4832065299011),
 (931, 388.34408127201857),
 (1279, 388.0664997728012),
 (1302, 387.9280419921482),
 (1290, 387.789805171247)]